# Exercise 1 - Simple Data Parallel Example

**GOAL:** The goal of this exercise is to show how to run simple tasks in parallel.

The Python script used in this exercise runs too slowly, although its computation is embarrassingly parallel. Use Ray to execute the functions in parallel to speed up the script.

### Concept for this Exercise - Remote Functions

The standard way to turn a Python function into a *remote function* is to add the `@ray.remote` decorator. Here is an example.

```python
# A regular Python function.
def regular_function():
    return 1

# A Ray remote function.
@ray.remote
def remote_function():
    return 1
```

The differences are the following:

1. **Invocation:** The regular version is called with `regular_function()`, whereas the remote version is called with `remote_function.remote()`.
2. **Return values:** `regular_function` immediately executes and returns `1`, whereas `remote_function` immediately returns an *object ref* (a future) and then creates a task that will be executed on a worker process. The result can be obtained with `ray.get`.
    ```python
    >>> regular_function()
    1
    
    >>> remote_function.remote()
    ObjectID(1c80d6937802cd7786ad25e50caf2f023c95e350)
    
    >>> ray.get(remote_function.remote())
    1
    ```
3. **Parallelism:** Invocations of `regular_function` happen **serially**, for example
    ```python
    # These happen serially.
    for _ in range(4):
        regular_function()
    ```
    whereas invocations of `remote_function` happen in **parallel**, for example
    ```python
    # These happen in parallel.
    for _ in range(4):
        remote_function.remote()
    ```

This example needs to run four tasks concurrently.
To illustrate how these tasks run in parallel, we'll tell Ray explicitly that there are four CPUs.

Usually this is not done.
By default, Ray does not schedule more tasks concurrently than there are CPUs. 
Instead it computes the number of CPUs using `psutil.cpu_count()`:

In [ ]:
import psutil

psutil.cpu_count()

First, let's start Ray using `ray.init` -- with the `num_cpus` set explicitly.
This starts a number of processes. 
The other argument `ignore_reinit_error=True` simply ignores errors if this cell gets re-run multiple times.

In [ ]:
import ray
import time

ray.init(num_cpus=4, ignore_reinit_error=True)

**EXERCISE:** The function below run too slow. Turn it into a remote function using the `@ray.remote` decorator.

In [ ]:
# This function is a proxy for a more interesting and computationally
# intensive function.
def slow_function(i):
    time.sleep(1)
    return i

**EXERCISE:** The loop below takes too long. The four function calls could be executed in parallel.
Instead of four seconds, it should only take one second for total run time.

Once `slow_function` has been made a remote function, execute these four tasks in parallel by calling `slow_function.remote()`.
Then obtain the results by calling `ray.get` on a list of the resulting object refs.

In [ ]:
# Sleep a little to improve the accuracy of the timing measurements below.
# We do this because workers may still be starting up in the background.
time.sleep(2.0)
start_time = time.time()

results = [slow_function(i) for i in range(4)]
    
end_time = time.time()
duration = end_time - start_time

print('The results are {}. This took {} seconds. Run the next cell to see '
      'if the exercise was done correctly.'.format(results, duration))

**VERIFY:** Run some checks to verify that the changes you made to the code were correct. Some of the checks should fail when you initially run the cells. After completing the exercises, the checks should pass.

In [ ]:
assert results == [0, 1, 2, 3], 'Did you remember to call ray.get?'
assert duration < 1.1, ('The loop took {} seconds. This is too slow.'
                        .format(duration))
assert duration > 1, ('The loop took {} seconds. This is too fast.'
                      .format(duration))

print('Success! The example took {} seconds.'.format(duration))

**EXERCISE:** Use the UI to view the task timeline and to verify that the four tasks were executed in parallel. You can do this as follows.

1. Run the following cell to generate a JSON file containing the profiling data.
2. Download the timeline file by right clicking on `timeline01.json` in the navigator to the left and choosing **"Download"**.
3. Open [`chrome://tracing/`](chrome://tracing/) in the Chrome web browser, click on the **"Load"** button and load the downloaded JSON file.

To navigate within the timeline, do the following.
- Move around by clicking and dragging.
- Zoom in and out by holding **alt** (or **option**) and scrolling.

**NOTE:** The timeline visualization will only work in **Chrome**.

In [ ]:
ray.timeline(filename="timeline01.json")